Let's import the agent SDK classes and functions we need, as long as sendgrid helper info.

In [3]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [4]:
load_dotenv(override=True)

True

### Email Function
We could use this to allow the LLM to email us for a variet of reasons. For example, maybe it wants to tell us there's a question it couldn't answer. Another example might be if someone is reaching out to us and the LLM wants to say something like, "Hey, Jonas from Masifunde is asking to connect."

In [5]:

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("jim@masinyusane.org")
    to_email = To("jim@masinyusane.org")
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

UnauthorizedError: HTTP Error 401: Unauthorized

This is some background info for us to dynamically plug into each of the agent contexts

In [20]:
zz_background = """# PROGRAMME OVERVIEW
- Programme: **Zazi iZandi** (South Africa)
- Intervention: Teaching small groups of **7 children** their letter sounds in a **frequency‑based sequence**.
- Groups are **level‑based**: each group may be working on different letters at any given time.
- Teacher Assistants (TAs) use an official **Letter Tracker** ordered by letter frequency."""

Let's create some instructions for our agents. Putting this here to make the code more readable.

In [22]:
instructions_2023 = f"You are a helpful data analyst. You help the user with understanding the performance of the Zazi iZandi literacy programme in 2023. {zz_background}"
instructions_2024 = f"You are a helpful data analyst. You help the user with understanding the performance of the Zazi iZandi literacy programme in 2024. {zz_background}"
instructions_2025 = f"You are a helpful data analyst. You help the user with understanding the performance of the Zazi iZandi literacy programme in 2025. {zz_background}"


Okay, let's create some agents!

In [23]:
zazi_2023_agent = Agent(
        name="Zazi 2023 Agent",
        instructions=instructions_2023,
        model="gpt-4o-mini"
)

zazi_2024_agent = Agent(
        name="Zazi 2024 Agent",
        instructions=instructions_2024,
        model="gpt-4o-mini"
)

zazi_2025_agent = Agent(
        name="Zazi 2025 Agent",
        instructions=instructions_2025,
        model="gpt-4o-mini"
)

Now let's test the first agent and see that it runs.

In [24]:
result = Runner.run_streamed(zazi_2023_agent, input="How many children were on the programme in 2023?")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

To determine how many children were in the Zazi iZandi literacy programme in 2023, I would need specific enrollment data or statistics provided for that year. If you have any figures or additional details regarding the number of groups or total participants, please share them, and I can help you analyze the data.

Okay, now let's pull in some data to give the agents some basic tools. Just going to set root path and then import some of the data loaders & preprocessors

In [11]:
import os
import sys
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import streamlit as st

# We need to set root directory so we can find the zz_data_process_23.py file. I should obviously move this into a better named directory, will do so in the future.
root_dir = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if root_dir not in sys.path:
    sys.path.append(root_dir)

#2023 Data
from zz_data_process_23 import process_zz_data_23
from data_loader import load_zazi_izandi_2023

#2024 Data
from zz_data_processing import process_zz_data_midline, process_zz_data_endline, grade1_df, gradeR_df
from data_loader import load_zazi_izandi_2024


Create functions to import 2023 and 2024 data.

In [12]:
    
def import_2023_results():
    # Import dataframes
    endline_df, sessions_df = load_zazi_izandi_2023()
    endline = process_zz_data_23(endline_df, sessions_df)
    return endline

In [13]:
def import_2024_results():
    baseline_df, midline_df, sessions_df, baseline2_df, endline_df, endline2_df = load_zazi_izandi_2024()
    
    # Create deep copies to ensure data independence between tabs
    baseline_df = baseline_df.copy()
    midline_df = midline_df.copy()
    sessions_df = sessions_df.copy()
    endline_df = endline_df.copy()

    midline, baseline = process_zz_data_midline(baseline_df, midline_df, sessions_df)
    endline = process_zz_data_endline(endline_df)
    grade1 = grade1_df(endline)
    gradeR = gradeR_df(endline)
    
    return endline

Okay, let's create a super simple tool.

In [15]:
@function_tool
def get_2023_number_of_children():
    """
    Get the number of children on the programme in 2023
    """
    endline_2023 = import_2023_results()
    number_of_children = len(endline_2023)
    return number_of_children

@function_tool
def get_2024_number_of_children():
    """
    Get the number of children on the programme in 2024
    """
    endline_2024 = import_2024_results()
    number_of_children = len(endline_2024)
    return number_of_children

Let's update the instructions to tell the LLM to use the tool to get the number of children if asked.

In [25]:
instructions_2023 = f"You are a helpful data analyst. You help the user with understanding the performance of the Zazi iZandi literacy programme in 2023. {zz_background}. If you need to know the number of children on the programme, you can use the get_2023_number_of_children function."
instructions_2024 = f"You are a helpful data analyst. You help the user with understanding the performance of the Zazi iZandi literacy programme in 2024. {zz_background}. If you need to know the number of children on the programme, you can use the get_2024_number_of_children function."
instructions_2025 = f"You are a helpful data analyst. You help the user with understanding the performance of the Zazi iZandi literacy programme in 2025. {zz_background}. If you need to know the number of children on the programme, you can use the get_2025_number_of_children function."


Now let's update the agents to have the tools we created.

In [26]:
zazi_2023_agent = Agent(
        name="Zazi 2023 Agent",
        instructions=instructions_2023,
        model="gpt-4o-mini",
        tools=[get_2023_number_of_children]
)

zazi_2024_agent = Agent(
        name="Zazi 2024 Agent",
        instructions=instructions_2024,
        model="gpt-4o-mini",
        tools=[get_2024_number_of_children]
)

In [27]:
question = "How many children were on the programme in 2023?"

In [28]:
with trace("Zazi 2023 Agent"):
    result = await Runner.run(zazi_2023_agent, question)

print(result.final_output)

2025-09-12 09:18:38.561 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
/Users/jimmckeown/Python/ZZ Data Site/venv/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In 2023, there were 1,896 children on the Zazi iZandi literacy programme.


Below is a version of the 2023 agent streaming back the results.

In [29]:
result = Runner.run_streamed(zazi_2023_agent, input=question)
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

2025-09-12 09:18:43.693 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
/Users/jimmckeown/Python/ZZ Data Site/venv/lib/python3.13/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In 2023, there were **1,896 children** on the Zazi iZandi literacy programme.

# Next Steps
- Create an agentic solution where a supervisor agent can use the 2023 and 2024 agents to answer questions.